In [ ]:
from ar6_ch6_rcmipfigs.constants import INPUT_DATA_DIR

__depends__ = []
__dest__ = [INPUT_DATA_DIR+
    "/data/database-results/phase-1/timestamp.txt",
    INPUT_DATA_DIR+"/data/database-observations/timestamp.txt",
]

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [24]:
#TODO: clean up more


<IPython.core.display.Javascript object>

# Database generation

In this notebook we process the data into a database we can later query to make plots/do analysis etc.

## Imports

In [3]:
import logging
import os.path
import re
from pathlib import Path
from distutils.util import strtobool

import pandas as pd
import pyam
import tqdm
from scmdata import ScmDataFrame, df_append

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
from ar6_ch6_rcmipfigs.utils.database_generation import check_all_variables_and_units_as_in_protocol, \
    check_all_scenarios_as_in_protocol, unify_units, save_into_database, mce_get_quantile, hector_get_quantile

TEST_RUN = strtobool(os.getenv("CI", "False")) or False
TEST_RUN

False

<IPython.core.display.Javascript object>

In [5]:
logger = logging.getLogger()

<IPython.core.display.Javascript object>

## Constants

In [6]:
from ar6_ch6_rcmipfigs.constants import  INPUT_DATA_DIR
OUTPUT_DATABASE_PATH = os.path.join(INPUT_DATA_DIR, "database-results", "phase-1/")

OBS_DATABASE_PATH = os.path.join(INPUT_DATA_DIR, "database-observations/")

<IPython.core.display.Javascript object>

In [7]:
from ar6_ch6_rcmipfigs.utils.misc import make_folders
if not os.path.isdir(OUTPUT_DATABASE_PATH):
    make_folders(OUTPUT_DATABASE_PATH)

if not os.path.isdir(OBS_DATABASE_PATH):
    make_folders(OBS_DATABASE_PATH)

<IPython.core.display.Javascript object>

## Protocol

In [8]:
SCENARIO_PROTOCOL = os.path.join(INPUT_DATA_DIR, "data", "protocol", "rcmip-emissions-annual-means.csv"
)

<IPython.core.display.Javascript object>

In [9]:
protocol_db = ScmDataFrame(SCENARIO_PROTOCOL)
protocol_db.head()

time                                                                               1750-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1751-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1752-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1753-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1754-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable CMIP5                    NaN   
                      Emissions|CH4              Mt CH4/yr not_applicable CMIP5                    NaN   
                      Emissions|CO               Mt CO/yr  not_applicable CMIP5                    NaN   
                      Emissions|CO2              Mt CO2/yr not_applicable CMIP5                    NaN   
                      Emissions|CO2|MAGICC AFOLU Mt CO2/yr not_applicable CMIP5                    NaN   

time                                                                               1755-01-01 00:00:00  \
model scenario region variable                   unit      activity_id    mip_era                        
AIM   rcp60    World  Emissions|BC               Mt BC/yr  not_applicable

<IPython.core.display.Javascript object>

In [10]:
protocol_db["scenario"].unique()

array(['rcp60', 'ssp370', 'ssp370-lowNTCF-aerchemmip',
       'ssp370-lowNTCF-gidden', 'historical', 'ssp434', 'ssp460', 'rcp26',
       'ssp119', 'ssp126', 'rcp85', 'ssp245', 'rcp45', 'ssp534-over',
       'ssp585', 'esm-bell-1000PgC', 'esm-bell-2000PgC',
       'esm-bell-750PgC', 'esm-pi-CO2pulse', 'esm-pi-cdr-pulse',
       'esm-piControl', 'historical-cmip5'], dtype=object)

<IPython.core.display.Javascript object>

In [11]:
DATA_PROTOCOL = os.path.join(INPUT_DATA_DIR,
    "data",
    "submission-template",
    "rcmip-data-submission-template.xlsx",
)

<IPython.core.display.Javascript object>

In [12]:
protocol_variables = pd.read_excel(DATA_PROTOCOL, sheet_name="variable_definitions")
protocol_variables.columns = protocol_variables.columns.str.lower()
protocol_variables.head()

,idx,category,variable,unit,definition,tier
0,1,Atmospheric Concentrations,Atmospheric Concentrations|CH4,ppb,atmospheric concentrations of CH4,1
1,2,Atmospheric Concentrations,Atmospheric Concentrations|CO2,ppm,atmospheric concentrations of CO2,1
2,3,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases,ppm,equivalent species atmospheric concentrations ...,3
3,4,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases|HFC,ppm,equivalent species atmospheric concentrations ...,3
4,5,Atmospheric Concentrations,Atmospheric Concentrations|F-Gases|HFC|HFC125,ppt,atmospheric concentrations of HFC125,2


<IPython.core.display.Javascript object>

In [13]:
protocol_scenarios = pd.read_excel(
    DATA_PROTOCOL, sheet_name="scenario_info", skip_rows=2
)
protocol_scenarios.columns = protocol_scenarios.columns.str.lower()
protocol_scenarios.head()

,unnamed: 0,scenario,description of scenario,detailed scenario description,priority
0,Instructions/Details,Scenario identifier used in the CMIP6 modeling...,Brief description of scenario,Documentation on es-doc + some other clarifyin...,"Top priority is Tier 1, additional runs are Ti..."
1,Answers,# Scenario ID,# Scenario Description,#Scenario Specification,# Tier in RCMP
2,NaN,piControl,pre-industrial control simulation (i.e. consta...,groups should run as long as is sensible/they ...,1
3,NaN,esm-piControl,pre-industrial control simulation with zero an...,should be identical to piControl except CO2 is...,1
4,NaN,esm-piControl-allGHG,pre-industrial control simulation with zero an...,should be identical to piControl except all GH...,2


<IPython.core.display.Javascript object>

## Observations

### tas observations

These come from Chris Smith (personal email). TODO: get details from Chris about what these are.

In [14]:
TAS_OBS_PATH = os.path.join(
    INPUT_DATA_DIR, "data", "priestley-centre", "observations", "tas_obs.txt"
)

<IPython.core.display.Javascript object>

In [15]:
tas_obs_df = pd.read_csv(TAS_OBS_PATH, header=None, delim_whitespace=True)
tas_obs_df.columns = ["year", "value"]
tas_obs_df["model"] = "unspecified"
tas_obs_df["climatemodel"] = "Observations (Priestley Centre)"
tas_obs_df["scenario"] = "historical"
tas_obs_df["variable"] = "Surface Air Temperature Change"
tas_obs_df["unit"] = "K"
tas_obs_df["region"] = "World"
tas_obs_df = pyam.IamDataFrame(tas_obs_df)
tas_obs_df.head()

,model,scenario,region,variable,unit,year,climatemodel,value
0,unspecified,historical,World,Surface Air Temperature Change,K,1850.0,Observations (Priestley Centre),-0.042412
1,unspecified,historical,World,Surface Air Temperature Change,K,1851.0,Observations (Priestley Centre),0.123588
2,unspecified,historical,World,Surface Air Temperature Change,K,1852.0,Observations (Priestley Centre),0.085588
3,unspecified,historical,World,Surface Air Temperature Change,K,1853.0,Observations (Priestley Centre),0.025588
4,unspecified,historical,World,Surface Air Temperature Change,K,1854.0,Observations (Priestley Centre),0.086588


<IPython.core.display.Javascript object>

In [16]:
db_obs = pyam.concat([tas_obs_df])
db_obs.head()

,model,scenario,region,variable,unit,year,climatemodel,value
0,unspecified,historical,World,Surface Air Temperature Change,K,1850.0,Observations (Priestley Centre),-0.042412
1,unspecified,historical,World,Surface Air Temperature Change,K,1851.0,Observations (Priestley Centre),0.123588
2,unspecified,historical,World,Surface Air Temperature Change,K,1852.0,Observations (Priestley Centre),0.085588
3,unspecified,historical,World,Surface Air Temperature Change,K,1853.0,Observations (Priestley Centre),0.025588
4,unspecified,historical,World,Surface Air Temperature Change,K,1854.0,Observations (Priestley Centre),0.086588


<IPython.core.display.Javascript object>

In [17]:
save_into_database(db_obs, OBS_DATABASE_PATH, "rcmip-observations")

<IPython.core.display.Javascript object>

## Model output

In [18]:
RESULTS_PATH = os.path.join(INPUT_DATA_DIR, "data", "results", "phase-1")

<IPython.core.display.Javascript object>

In [19]:
_results_files = list(Path(RESULTS_PATH).rglob("*.csv")) + list(
    Path(RESULTS_PATH).rglob("*.xlsx")
)
print(len(_results_files))
sorted(_results_files)

453


[PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/acc2/rcmip_phase-1_acc2_v1-0-0.xlsx'),
 PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/acc2/rcmip_phase-1_acc2_v1-0-1.xlsx'),
 PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/acc2/rcmip_phase-1_acc2_v2-0-0.xlsx'),
 PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/acc2/rcmip_phase-1_acc2_v2-0-1.xlsx'),
 PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/ar5ir/ar5ir-phase-1-results-v1-0-0.csv'),
 PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/ar5ir/ar5ir-phase-1-results-v1-0-1.csv'),
 PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/ar5ir/ar5ir

<IPython.core.display.Javascript object>

In [20]:
model_of_interest = [
#    ".*acc2.*v2-0-1.*",
    ".*rcmip_phase-1_cicero-scm.*v5-0-0.*",
#    ".*escimo.*v2-0-1.*",
    ".*fair-1.5-default.*v1-0-1.csv",
#    ".*rcmip_phase-1_gir.*",
#    ".*greb.*v2-0-0.*",
#    ".*hector.*v2-0-0.*",
#    ".*MAGICC7.1.0aX-rcmip-phase-1.*",
    ".*rcmip_phase-1_magicc7.1.0.beta_v1-0-0.*",
#    ".*MAGICC7.1.0aX.*",
#    ".*mce.*v2-0-1.*",
#    ".*oscar-v3-0*v1-0-1.*",
    ".*oscar-v3-0.*v1-0-1.*"
#    ".*wasp.*v1-0-1.*",
]

<IPython.core.display.Javascript object>

In [21]:
if TEST_RUN:
    model_of_interest = [
        ".*escimo-phase-1-v2-0-1.*",
        ".*greb.*",
        ".*rcmip_phase-1_cicero-scm.*v5-0-0.*",
    ]

results_files = [
    str(p)
    for p in _results_files
    if any([bool(re.match(m, str(p))) for m in model_of_interest]) and "$" not in str(p)
]
print(len(results_files))
sorted(results_files)

80


['/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/cicero-scm/rcmip_phase-1_cicero-scm-ecs3_v5-0-0.csv',
 '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/cicero-scm/rcmip_phase-1_cicero-scm_v5-0-0.csv',
 '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-4xext_v1-0-1.csv',
 '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-bgc_v1-0-1.csv',
 '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-cdr_v1-0-1.csv',
 '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/fair/rcmip_phase-1_fair-1.5-default-1pctCO2-rad_v1-0-1.csv',
 '/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/d

<IPython.core.display.Javascript object>

In [22]:
[
    str(p)
    for p in results_files
    if 'magicc' in str(p)] #for m in model_of_interest]) and "$" not in str(p)
#]

['/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGS/ar6_ch6_rcmipfigs/data_in/data/results/phase-1/magicc7/rcmip_phase-1_magicc7.1.0.beta_v1-0-0.csv']

<IPython.core.display.Javascript object>

In [23]:
db = []
for rf in tqdm.tqdm_notebook(results_files):
    if rf.endswith(".csv"):
        loaded = ScmDataFrame(rf)
    else:
        loaded = ScmDataFrame(rf, sheet_name="your_data")
    db.append(loaded)

db = df_append(db).timeseries().reset_index()
db["unit"] = db["unit"].apply(
    lambda x: x.replace("Dimensionless", "dimensionless") if isinstance(x, str) else x
)
db = ScmDataFrame(db)
db.head()

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [ ]:
db.filter(climatemodel="*cicero*").head()

In [ ]:
db["climatemodel"].unique()

### Minor quick fixes

We relabel all the ssp370-lowNTCF data to remove ambiguity.

In [ ]:
db = db.timeseries().reset_index()
db["scenario"] = db["scenario"].apply(
    lambda x: "ssp370-lowNTCF-gidden" if x == "ssp370-lowNTCF" else x
)
db["scenario"] = db["scenario"].apply(
    lambda x: "esm-ssp370-lowNTCF-gidden" if x == "esm-ssp370-lowNTCF" else x
)
db["scenario"] = db["scenario"].apply(
    lambda x: "esm-ssp370-lowNTCF-gidden-allGHG"
    if x == "esm-ssp370-lowNTCF-allGHG"
    else x
)
db = ScmDataFrame(db)

In [ ]:
assert "ssp370-lowNTCF" not in db["scenario"].unique().tolist()
assert "esm-ssp370-lowNTCF" not in db["scenario"].unique().tolist()
assert "esm-ssp370-lowNTCF-allGHG" not in db["scenario"].unique().tolist()

The Hector and MCE data is mislabelled so we do a quick fix here. I also have changed my mind about how to format the quantiles so tweak the FaIR and WASP data too. TODO: email modelling groups so they can fix it for phase 2.

In [ ]:
mce_prob_data = db.filter(climatemodel="MCE*PROB*")
mce_prob_data["climatemodel"].unique()
if not mce_prob_data.timeseries().empty:
    mce_prob_data = mce_prob_data.timeseries().reset_index()

    mce_prob_data["variable"] = (
        mce_prob_data["variable"]
        + "|"
        + mce_prob_data["climatemodel"].apply(mce_get_quantile)
        + "th quantile"
    )

    mce_prob_data["climatemodel"] = mce_prob_data["climatemodel"].apply(
        lambda x: "-".join(x.split("-")[:-1])
    )

    db = db.filter(climatemodel="MCE*PROB*", keep=False).append(mce_prob_data)

db.filter(climatemodel="MCE*PROB").head(10)

In [ ]:
hector_prob_data = db.filter(climatemodel="hector*HISTCALIB*")
if not hector_prob_data.timeseries().empty:
    hector_prob_data = hector_prob_data.timeseries().reset_index()

    hector_prob_data["variable"] = (
        hector_prob_data["variable"]
        + "|"
        + hector_prob_data["climatemodel"].apply(hector_get_quantile)
    )

    hector_prob_data["climatemodel"] = hector_prob_data["climatemodel"].apply(
        lambda x: x.split("-")[0]
    )

    db = db.filter(climatemodel="hector*HISTCALIB*", keep=False).append(
        hector_prob_data
    )

db.filter(climatemodel="hector*HISTCALIB").head(10)

In [ ]:
fair_prob_data = db.filter(climatemodel="*FaIR*")
if not fair_prob_data.timeseries().empty:
    fair_prob_data = fair_prob_data.timeseries().reset_index()

    fair_prob_data["variable"] = fair_prob_data["variable"].apply(
        lambda x: x.replace("|00th", "|0th").replace("|05th", "|5th")
    )

    db = db.filter(climatemodel="*FaIR*", keep=False).append(
        ScmDataFrame(fair_prob_data)
    )

db.filter(climatemodel="*FaIR*").head(10)

In [ ]:
wasp_prob_data = db.filter(climatemodel="*WASP*")
if not wasp_prob_data.timeseries().empty:
    wasp_prob_data = wasp_prob_data.timeseries().reset_index()

    wasp_prob_data["variable"] = wasp_prob_data["variable"].apply(
        lambda x: x.replace("|00th", "|0th").replace("|05th", "|5th")
    )

    db = db.filter(climatemodel="*WASP*", keep=False).append(
        ScmDataFrame(wasp_prob_data)
    )

db.filter(climatemodel="*WASP*").head(10)

## Unify units and check names

Here we loop over the submissions and unify their units as well as checking their naming matches what we expect.

In [ ]:
base_df = db.timeseries()
any_failures = False

clean_db = []
for climatemodel, cdf in tqdm.tqdm_notebook(
    base_df.groupby("climatemodel"), desc="Climate model"
):
    print(climatemodel)
    print("-" * len(climatemodel))

    any_failures_climatemodel = False

    cdf = ScmDataFrame(cdf)
    cdf_converted_units = unify_units(cdf, protocol_variables)
    try:
        check_all_scenarios_as_in_protocol(cdf_converted_units, protocol_scenarios)
        check_all_variables_and_units_as_in_protocol(
            cdf_converted_units, protocol_variables
        )
    except AssertionError as e:
        print(e)
        any_failures_climatemodel = True
    #     # currently not possible as groups weren't told to obey variable hierarchy,
    #     # add this in phase 2
    #     for v_top in cdf_converted_units.filter(level=0)["variable"].unique():
    #         print(v_top)
    #         cdf_pyam = cdf_converted_units.filter(variable="{}*".format(v_top)).timeseries()
    #         cdf_pyam.columns = cdf_pyam.columns.map(lambda x: x.year)

    #         cdf_consistency_checker = pyam.IamDataFrame(cdf_pyam)
    #         if cdf_consistency_checker.check_internal_consistency() is not None:
    #             print("Failed for {}".format(v_top))
    #             any_failures_climatemodel = True
    #             failing_set = cdf_consistency_checker.copy()

    print()
    if not any_failures_climatemodel:
        clean_db.append(cdf_converted_units)
        print("All clear for {}".format(climatemodel))
    else:
        print("Failed {}".format(climatemodel))
        print("X" * len("Failed"))
        any_failures = True

    print()
    print()

if any_failures:
    raise AssertionError("database isn't ready yet")
else:
    clean_db = df_append(clean_db)
    clean_db.head()

In [ ]:
clean_db.head()

Notes whilst doing this:

- I wasn't clear that the variable hierarchy needs to be obeyed, hence doing internal consistency checks isn't going to work

For phase 2:

- checking internal consistency super slow, worth looping over top level variables when doing this to speed up filtering
- need to decide what a sensible tolerance is
- might have to go back to model notes to work out why there are inconsistencies
- will have to implement a custom hack to deal with the double counting in the direct aerosol forcing hierarchy

## Creating a database

In [ ]:
save_into_database(clean_db, OUTPUT_DATABASE_PATH, "rcmip-phase-1")